In [22]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects.aio import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, MCPTool, Tool, Connection
import os
import aiohttp

In [ ]:
load_dotenv(override=True)

AI_FOUNDRY_ENDPOINT=os.getenv('AI_FOUNDRY_ENDPOINT')
AI_FOUNDRY_MODEL=os.getenv('AI_FOUNDRY_MODEL')
MCP_MENU_CONN_NAME = os.getenv('MCP_MENU_CONNECTION_NAME')
MCP_MENU_ENDPOINT=os.getenv('MCP_MENU_URL')
MCP_ORDER_CONN_NAME=os.getenv('MCP_ORDER_CONNECTION_NAME')
MCP_ORDER_ENDPOINT=os.getenv('MCP_ORDER_URL')

In [10]:
project = AIProjectClient(
    endpoint=AI_FOUNDRY_ENDPOINT,
    credential=DefaultAzureCredential()
)

client = project.get_openai_client()

In [11]:
response = await client.responses.create(
    model=AI_FOUNDRY_MODEL,
    input="What is the best meal in Quebec ?"
)

In [ ]:
response.output_text

In [13]:
results = project.agents.list()

async for agent in results:
    print(agent.name)

In [ ]:
results = project.connections.list()
connection_id:str = None 

async for conn in results:
    if conn.name == MCP_MENU_CONN_NAME:
        connection_id = conn.id

print(connection_id)

In [29]:
menu_mcp_server = MCPTool(
    server_label='MCPMenuServer',
    server_url=MCP_MENU_ENDPOINT,
    require_approval='never',
    project_connection_id="MCPMenuServer"
)

In [34]:
INSTRUCTION="""
You are a menu assistant for Contoso Pizza, you only call the tool MCPMenuServer to answer question from the user.  
If the user want to see the complete menu, just don't pass the parameter itemType.
If question are not related to ordering menu from Contoso Pizza, you don't create any response. 
"""

In [ ]:
await project.agents.create_version(
    agent_name="MenuAgent",
    definition=PromptAgentDefinition(
        model=AI_FOUNDRY_MODEL,
        instructions=INSTRUCTION,
        tools=[menu_mcp_server]
    )
)

In [ ]:
order_mcp_server = MCPTool(
    server_label='MCPMenuServer',
    server_url=MCP_MENU_ENDPOINT,
    require_approval='never',
    project_connection_id="MCPMenuServer"
)